This quickstart tutorial will walk you through how to use an Extended Mind Transformer

Let's start with a simple question-answer example, and see if we can eliminate a simple model hallucination.

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("normalcomputing/extended-mind-mpt-7b")
model = AutoModelForCausalLM.from_pretrained("normalcomputing/extended-mind-mpt-7b", trust_remote_code=True).to(device)

In [ ]:
query = "When did Alexander Grothendieck get his French citizenship?"
inputs = tokenizer(query, return_tensors="pt").input_ids

outputs_hf = model.generate(inputs.to(device), max_length= inputs.size(-1) + 40,)
print(tokenizer.decode(outputs_hf[0], skip_special_tokens=True))

Oops! Not quite right. Now, let's give the model the information it needs as memories. We can pass memories in the `from_pretrained()` method, or easily set them afterwards as below.

In [ ]:
ag_wiki_entry = """Alexander Grothendieck (/ˈɡroʊtəndiːk/; German pronunciation: [ˌalɛˈksandɐ ˈɡʁoːtn̩ˌdiːk] (listen); French: [ɡʁɔtɛndik]; 28 March 1928 – 13 November 2014) was a stateless (and then, since 1971, French) mathematician who became the leading figure in the creation of modern algebraic geometry.[7][8] His research extended the scope of the field and added elements of commutative algebra, homological algebra, sheaf theory, and category theory to its foundations, while his so-called "relative" perspective led to revolutionary advances in many areas of pure mathematics.[7][9] He is considered by many to be the greatest mathematician of the twentieth century.[10][11]

Grothendieck began his productive and public career as a mathematician in 1949. In 1958, he was appointed a research professor at the Institut des hautes études scientifiques (IHÉS) and remained there until 1970, when, driven by personal and political convictions, he left following a dispute over military funding. He received the Fields Medal in 1966 for advances in algebraic geometry, homological algebra, and K-theory.[12] He later became professor at the University of Montpellier[1] and, while still producing relevant mathematical work, he withdrew from the mathematical community and devoted himself to political and religious pursuits (first Buddhism and later, a more Christian vision).[13] In 1991, he moved to the French village of Lasserre in the Pyrenees, where he lived in seclusion, still working tirelessly on mathematics and his philosophical and religious thoughts until his death in 2014.[14]
"""
memories = tokenizer(ag_wiki_entry).input_ids
model.memory_ids = memories

In [ ]:
outputs_hf = model.generate(inputs.to(device), max_length=inputs.size(-1) + 15, topk=8)
print(tokenizer.decode(outputs_hf[0], skip_special_tokens=True))

Choosing the number of memories the model retrieves (`k` above) is an important hyperparameter. As we increase `topk`, the model retrieves more memories. Let's see how the generations evolve as we increase `topk`. 

In [ ]:
out = model.generate(inputs.to(model.device), max_length=inputs.size(-1) + 15, topk=5)
print("Generation for k=5: ", tokenizer.decode(out[0][inputs.size(-1) :]).strip())

out = model.generate(inputs.to(model.device), max_length=inputs.size(-1) + 15, topk=6)

print("Generation for k=6: ", tokenizer.decode(out[0][inputs.size(-1) :]).strip())

out = model.generate(inputs.to(model.device), max_length=inputs.size(-1) + 20, topk=7)
print("Generation for k=7: ", tokenizer.decode(out[0][inputs.size(-1) :]).strip())

out = model.generate(inputs.to(model.device), max_length=inputs.size(-1) + 15, topk=8)
print("Generation for k=8: ", tokenizer.decode(out[0][inputs.size(-1) :]).strip())

out = model.generate(inputs.to(model.device), max_length=inputs.size(-1) + 20, topk=30)
print("Generation for k=30: ", tokenizer.decode(out[0][inputs.size(-1) :]).strip())
